<a href="https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age.ipynb


In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle 

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!kaggle datasets list

In [ ]:
!kaggle datasets download -d kmader/rsna-bone-age -p ./

100% 9.28G/9.29G [03:38<00:00, 41.4MB/s]
100% 9.29G/9.29G [03:38<00:00, 45.7MB/s]


In [ ]:
!unzip rsna-bone-age.zip

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: boneage-training-dataset/boneage-training-dataset/4264.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4265.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4266.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4268.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4269.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4270.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4271.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4272.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4273.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4275.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4276.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4277.png  
  inflating: boneage-training-dataset/boneage

In [ ]:
import os
from PIL import Image

size=224
input_dir = "/content/boneage-training-dataset/boneage-training-dataset"
dirs = os.listdir(input_dir)
for item in dirs:
  image_path = os.path.join(input_dir, item)
  if os.path.isfile(image_path) and image_path.endswith('.png'):
    im = Image.open(image_path)
    imResize = im.resize((size,size), Image.ANTIALIAS)
    imResize.save(image_path, 'PNG', quality=90)
    print(image_path)

/content/boneage-training-dataset/boneage-training-dataset/6640.png
/content/boneage-training-dataset/boneage-training-dataset/9534.png
/content/boneage-training-dataset/boneage-training-dataset/2325.png
/content/boneage-training-dataset/boneage-training-dataset/12536.png
/content/boneage-training-dataset/boneage-training-dataset/13076.png
/content/boneage-training-dataset/boneage-training-dataset/10911.png
/content/boneage-training-dataset/boneage-training-dataset/10426.png
/content/boneage-training-dataset/boneage-training-dataset/12751.png
/content/boneage-training-dataset/boneage-training-dataset/7483.png
/content/boneage-training-dataset/boneage-training-dataset/8752.png
/content/boneage-training-dataset/boneage-training-dataset/14274.png
/content/boneage-training-dataset/boneage-training-dataset/5457.png
/content/boneage-training-dataset/boneage-training-dataset/12225.png
/content/boneage-training-dataset/boneage-training-dataset/8935.png
/content/boneage-training-dataset/boneage

In [ ]:
!rm rsna-bone-age.zip
!rm -r boneage-test-dataset
!rm boneage-test-dataset.csv

In [ ]:
!rm -r boneage-dataset-deit
!mkdir boneage-dataset-deit
!mkdir boneage-dataset-deit/train
!mkdir boneage-dataset-deit/val

In [ ]:
import pandas as pd
import shutil
import random

input_dir = "/content/boneage-training-dataset/boneage-training-dataset"
output_dir_train = "boneage-dataset-deit/train"
output_dir_val = "boneage-dataset-deit/val"
df = pd.read_csv('boneage-training-dataset.csv')

for index, row in df.iterrows():
  print(row['id'], row['boneage'], row['male'])
  
  file_path = os.path.join(input_dir, str(row['id']) + ".png")
  boneage = str(row['boneage'])

  output_subdir_train = os.path.join(output_dir_train, boneage)
  if not os.path.exists(output_subdir_train):
    os.mkdir(output_subdir_train)
  output_subdir_val = os.path.join(output_dir_val, boneage)
  if not os.path.exists(output_subdir_val):
    os.mkdir(output_subdir_val)

  if random.uniform(0, 1) <= 0.7:
    shutil.copy(file_path, output_subdir_train)
  else:
    shutil.copy(file_path, output_subdir_val)

In [ ]:
!git clone https://github.com/facebookresearch/deit.git

In [ ]:
%cd deit/

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install timm==0.3.2

In [19]:
!python main.py --model deit_small_patch16_224 --batch-size 64 --epochs 400 --output_dir /content/deit/output --data-path /content/boneage-dataset-deit

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Test:  [10/79]  eta: 0:00:21  loss: 3.5869 (3.5328)  acc1: 0.0000 (40.1515)  acc5: 0.0000 (40.1515)  time: 0.3126  data: 0.1810  max mem: 2492
Test:  [20/79]  eta: 0:00:13  loss: 2.5468 (3.1964)  acc1: 0.0000 (30.1587)  acc5: 100.0000 (62.4008)  time: 0.1413  data: 0.0148  max mem: 2492
Test:  [30/79]  eta: 0:00:09  loss: 2.6823 (3.2312)  acc1: 0.0000 (20.4301)  acc5: 77.0833 (49.9328)  time: 0.1431  data: 0.0189  max mem: 2492
Test:  [40/79]  eta: 0:00:07  loss: 2.6837 (3.1440)  acc1: 0.0000 (15.4472)  acc5: 68.7500 (59.9593)  time: 0.1418  data: 0.0188  max mem: 2492
Test:  [50/79]  eta: 0:00:05  loss: 2.6536 (3.1318)  acc1: 0.0000 (12.4183)  acc5: 100.0000 (55.9641)  time: 0.1478  data: 0.0240  max mem: 2492
Test:  [60/79]  eta: 0:00:03  loss: 3.2972 (3.4223)  acc1: 0.0000 (10.3825)  acc5: 0.0000 (46.7896)  time: 0.1447  data: 0.0218  max mem: 2492
Test:  [70/79]  eta: 0:00:01  loss: 4.6209 (3.6007)  acc1: 0.0000 (8.

In [ ]:
!python main.py --model deit_small_patch16_224 --batch-size 32 --epochs 600 --resume /content/deit/output/checkpoint.pth --output_dir /content/deit/output --data-path /content/boneage-dataset-deit

In [ ]:
!python main.py --eval --resume /content/deit/output/checkpoint.pth --model deit_small_patch16_224 --batch-size 32 --data-path /content/boneage-dataset-deit

In [ ]:
from PIL import Image
import torch
import timm

import torchvision.transforms as transforms

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

model = torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=False)
ckpt_path = '/content/deit/output/checkpoint.pth'

ckpt_dict = torch.load(ckpt_path)
model.load_state_dict(ckpt_dict["model"])

model.eval()

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),  # to maintain same ratio w.r.t. 224 images
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

img_path = '/content/dataset_3/test/capsella_bursa_pastoris/159.png'
#img_path = '/content/dataset_3/test/scentless_mayweed/10.png'
#img_path = '/content/dataset_3/test/small_flowered_cranesbill/1.png'
img = Image.open(img_path).convert("RGB")
img = transform(img)[None,]

out = model(img)
pred_label = torch.argmax(out)
print(pred_label.item())
